# Effective Sample Size (N_Effective)

## Effective Samples Introduction
From Wikipedia *In statistics and quantitative research methodology, a data sample is a set of data collected and/or selected from a statistical population by a defined procedure*. Properly sampling is one of the most challenging parts of experimental design for all statistics, one of the pitfalls being correlated data points in a sample.

For example if designing a drug to lower blood pressure, and if only able to 10 readings consider 3 designs.

1. 10 readings from 1 person
2. 10 readings from 10 people in one family
3. 10 readings from 10 random people

In each of these designs the number of data points remains the same, 10 readings, but an argument can be made that the **effective sample size** is not the same. In the first example, 10 readings from 1 person is a poor sample of if a drug works, there effectively was only one test if a drug had an effect. The second example is an improvement, but given knowledge of genetics family members will likely have similar reponses to a drug. The effective sample size is not one, but it could be argued that the sample size is greater than 10. In the last case 10 readings from 10 people would probably

The key notion is that effective samples are ones that present new information about the objective of the experiment.

## Effective Sample Size in Markov Chain Monte Carlo
In MCMC samples are draws from the Markov Chain. Before the chain reaches convergence draws could be correlated, meaning the nth sample may not be independent from the (n-1) draw.

To estimate the Gelman etal provide the following calculation $$\hat{n}_{eff} = \frac{mn}{1 + 2 \sum_{t=1}^T \hat{\rho}_t}$$

**TODO** Provide more information about effective sample size  



### Examples

In [1]:
import pystan
import arviz as az
import numpy as np

# Generate observed distribution with fixed parameters
SD = 2
MU = -5
obs = np.random.normal(loc=MU, scale=SD, size=10000)

In [2]:
import logging
import sys
logger = logging.getLogger()
logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

### TODO Create an example of model with a high number of effective samples, and a low number of effective samples

Trying to create a model thats initialized with poor priors to show that correlated draws and model parametrization affects the effective number of samples, even when the same number of draws are taken from an MCMC distribution

Open to any other construction of example if there's better ideas. Trying to shoot for simple models that are easily understood.

In [3]:
# Set prior mean to -5000, very far from simulated mean
stan_model = """
data {
    int<lower=0> N;
    real obs[N];
}
parameters {
    real mu;
}
model {
    mu ~ normal(-5000, 1);
    obs ~ normal(mu, 2);
}
"""

In [4]:
# fit = pystan.stan(model_code=stan_model, data={"obs":obs, "N":10000}, iter=500, chains=2, algorithm="NUTS", verbose=False)
# az.effective_n(fit)

In [6]:
# az.plot_autocorr(fit)

### TODO, create a model with less effective samples and explain why that happens

In [7]:
# Set prior mean to -5, exactly to prior mean
stan_model = """
data {
    int<lower=0> N;
    real obs[N];
}
parameters {
    real mu;
}
model {
    mu ~ normal(-5, 1);
    obs ~ normal(mu, 2);
}
"""

In [8]:
# fit = pystan.stan(model_code=stan_model, data={"obs":obs, "N":10000}, iter=500, chains=2, algorithm="NUTS", verbose=False)
# az.effective_n(fit)

# az.plot_autocorr(fit)

# Eight Schools Model

In [9]:
data = {
    "J": 8,
    "y": np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0]),
    "sigma": np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0]),
    }

In [11]:
non_centered_model = """
data {
  int<lower=0> J;
  real y[J];
  real<lower=0> sigma[J];
}

parameters {
  real mu;
  real<lower=0> tau;
  real theta[J];
}

model {
  mu ~ normal(0, 5);
  tau ~ cauchy(0, 5);
  theta ~ normal(mu, tau);
  y ~ normal(theta, sigma);
}
"""

In [12]:
fit = pystan.stan(model_code=non_centered_model, data=data, iter=500, chains=2, algorithm="NUTS", verbose=False)
az.effective_n(fit)

pystan - WARNING - DeprecationWarning: pystan.stan was deprecated in version 2.17 and will be removed in version 3.0. Compile and use a Stan program in separate steps.
pystan - INFO - COMPILING THE C++ CODE FOR MODEL anon_model_6e11919b5314121a277ebece94b64d05 NOW.
pystan - WARNING - Rhat for parameter mu is 1.1721458514295764!
pystan - WARNING - Rhat for parameter tau is 1.1202455952182722!
pystan - WARNING - Rhat for parameter theta[7] is 1.1314666640859392!
pystan - WARNING - Rhat for parameter lp__ is 1.3127727457822078!
pystan - WARNING - Rhat above 1.1 or below 0.9 indicates that the chains very likely have not mixed
pystan - WARNING - 100 of 500 iterations ended with a divergence (20.0%).
pystan - WARNING - Try running with adapt_delta larger than 0.8 to remove the divergences.
pystan - WARNING - Chain 1: E-BFMI = 0.17921493757693924
pystan - WARNING - E-BFMI below 0.2 indicates you may need to reparameterize your model


<xarray.Dataset>
Dimensions:      (theta_dim_0: 8)
Coordinates:
  * theta_dim_0  (theta_dim_0) int64 0 1 2 3 4 5 6 7
Data variables:
    mu           float64 7.221
    tau          float64 16.82
    theta        (theta_dim_0) float64 26.93 16.67 82.89 ... 10.51 12.92 25.24

# Centered Model

In [13]:
centered_model= """
data {
  int<lower=0> J;
  real y[J];
  real<lower=0> sigma[J];
}

parameters {
  real mu;
  real<lower=0> tau;
  real theta_tilde[J];
}

transformed parameters {
  real theta[J];
  for (j in 1:J)
    theta[j] = mu + tau * theta_tilde[j];
}

model {
  mu ~ normal(0, 5);
  tau ~ cauchy(0, 5);
  theta_tilde ~ normal(0, 1);
  y ~ normal(theta, sigma);
}
"""

In [14]:
fit = pystan.stan(model_code=centered_model, data=data, iter=500, chains=2, algorithm="NUTS", verbose=False)
az.effective_n(fit)

pystan - WARNING - DeprecationWarning: pystan.stan was deprecated in version 2.17 and will be removed in version 3.0. Compile and use a Stan program in separate steps.
pystan - INFO - COMPILING THE C++ CODE FOR MODEL anon_model_3f8f9e8bb354ab461436bb51d935571d NOW.
pystan - WARNING - 2 of 500 iterations ended with a divergence (0.4%).
pystan - WARNING - Try running with adapt_delta larger than 0.8 to remove the divergences.


<xarray.Dataset>
Dimensions:            (theta_dim_0: 8, theta_tilde_dim_0: 8)
Coordinates:
  * theta_dim_0        (theta_dim_0) int64 0 1 2 3 4 5 6 7
  * theta_tilde_dim_0  (theta_tilde_dim_0) int64 0 1 2 3 4 5 6 7
Data variables:
    mu                 float64 437.7
    tau                float64 394.5
    theta_tilde        (theta_tilde_dim_0) float64 486.0 376.5 ... 466.5 392.2
    theta              (theta_dim_0) float64 392.3 537.4 502.2 ... 436.2 388.9

## PyMC3 (To show what Im trying to get at)

In [ ]:
# Attempt to use pymc3 to estimate mean of the distribution
with pm.Model() as model:
    mu = pm.Normal("mu", mu=-5000, sd=1)
    y = pm.Normal("y", mu=mu, sd=SD, observed=obs)
    step = pm.Metropolis()
    trace = pm.sample(500, step, chains=2)
    
print(az.effective_n(trace))


# Attempt to use pymc3 to estimate mean of the distribution
with pm.Model() as model:
    mu = pm.Normal("mu", mu=-5, sd=1)
    y = pm.Normal("y", mu=mu, sd=SD, observed=obs)
    step = pm.Metropolis()
    trace = pm.sample(500, step, chains=2)
    
az.effective_n(trace)

#### The nice pystan output goes to the terminal (Possible to reverse outputs?)
<img src="PystanOutputInTerminal.png">

### Resources
[Effective Sample Size](https://www.youtube.com/watch?v=67zCIqdeXpo&t=227s) - Josh Starmer

### See also
Autocorrelation (TODO, Add link)

In [ ]:
%load_ext watermark

In [ ]:
%watermark -m

In [ ]:
%watermark